# New Red Bull API

For the Safari Rally Kenya, April 2025, a new API appears to have been introduced.

Now might be a good time to move everything into a database:

- all API calls are used to populate a database;
- all queries are made against the database;
- logic determines whether to upsert items to the database via new API calls.



In [ ]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

from urllib.parse import urljoin
import datetime
from sqlite_utils import Database
from jupyterlite_simple_cors_proxy.cacheproxy import CorsProxy, create_cached_proxy
import os
import sqlite3
from wrc_rallydj.db_table_schemas import SETUP_V2_Q
from pandas import read_sql, DataFrame, json_normalize, merge, concat

class WRCLiveTimingAPIClientV2:
    RED_BULL_LIVETIMING_API_BASE = (
        "https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/"
    )
    def __init__(
        self,
        year: int = datetime.date.today().year,
        championship: str = "wrc",
        group: str = "all",
        dbname: str = "wrcRbAPITiming.db",
        newDB: bool = False,
        use_cache: bool = False,
        **cache_kwargs,
    ):
        # Initialize the proxy with caching if requested
        if use_cache:
            self.proxy = create_cached_proxy(**cache_kwargs)
        else:
            self.proxy = CorsProxy()

        self.year = year
        self.championship = championship
        self.seasonId = None
        self.rallyId = None
        self.eventId = None

        # DB setup
        self.conn = None
        self.dbname = dbname
        # In the DB initialise phase also set self.seasonId
        self.setup_db(newdb=newDB)

    def initialise(self, year=None, championship=None):
        if year:
            self.year = year
        if championship:
            self.championship = championship
        # In the initialise phase, take the opportunity to update the season table
        self.seasonId = self._getSeasons(self.championship, self.year, updateDB=True).iloc[0][
            "seasonId"
        ]

    # db utils
    def cleardbtable(
        self,
        table,
        conn=None
    ):
        ''' Clear the table whilst retaining the table definition '''
        conn = conn if conn else self.conn

        c = conn.cursor()
        c.execute('DELETE FROM "{}"'.format(table))

    def dbfy(
        self,
        df,
        table,
        if_exists="upsert",
        pk=None,
        index=False,
        clear=False,
        conn=None,
        **kwargs,
    ):
        ''' Save a dataframe as a SQLite table.
            Clearing or replacing a table will first empty the table of entries but retain the structure. '''

        conn = conn if conn else self.conn

        # Upsert requires a PK
        if if_exists=="upsert" and not pk:
            return

        # print('{}: {}'.format(table,df.columns))
        if if_exists=='replace':
            clear=True
            if_exists='append'
        if clear:
            self.cleardbtable(table)

        # Get columns
        q="PRAGMA table_info({})".format(table)
        cols = read_sql(q, conn)['name'].tolist()
        for c in df.columns:
            if c not in cols:
                print('Hmmm... column name `{}` appears in data but not {} table def?'.format(c,table ))
                df.drop(columns=[c], inplace=True)

        logger.info(f"Updating {table}...")

        if if_exists=='upsert':
            DB = Database(conn)
            DB[table].upsert_all(df.to_dict(orient='records'), pk=pk)
        else:
            df.to_sql(table,conn,if_exists=if_exists,index=index)

    def setup_db(self, dbname=None, newdb=False):
        """Setup a database, if required, and return a connection."""
        logger.info("Initialising the database...")

        if dbname is not None:
            self.dbname = dbname
        dbname = self.dbname

        # In some situations, we may want a fresh start
        if os.path.isfile(dbname) and newdb:
            os.remove(dbname)

        if not os.path.isfile(dbname):
            # No db exists, so we need to create and populate one
            newdb = True

        # Open database connection
        self.conn = conn = sqlite3.connect(dbname, timeout=10)

        if newdb:
            logger.info("Creating new db tables...")
            # Setup database tables
            c = conn.cursor()
            c.executescript(SETUP_V2_Q)
            # c.executescript(SETUP_VIEWS_Q)

            # Populate the database with seasons info
            self._getSeasons(updateDB=True)

            # Initialise the seasonId
            self.seasonId = self._getSeasons(self.championship, self.year, updateDB=True).iloc[0][
                "seasonId"
            ]
            # Update the season detail
            self._getSeasonDetail(updateDB=True)

            # Populate the database with event metadata
            # self.dbfy(conn, getEventMetadata(), "event_metadata", if_exists="replace")

            # Get geo bits
            # kml_processor(meta["event_meta"])

            # Save the entry list, initial itinerary etc
            # _save_rally_base(meta, conn)

        return conn

    def _WRC_RedBull_json(self, path, base=None, retUrl=False):
        """Return JSON from API."""
        base = self.RED_BULL_LIVETIMING_API_BASE if base is None else base
        url = urljoin(base, path)
        print(url)
        if retUrl:
            return url
        # print(f"Fetching: {url}")
        try:
            r = self.proxy.cors_proxy_get(url)
        except:
            print("Error trying to load data.")
            return {}
        # r = requests.get(url)
        rj = r.json()
        if "status" in rj and rj["status"] == "Not Found":
            return {}
        return r.json()

    def _getSeasonsSubQuery(self, seasons_df, championship=None, year=None):
        if championship is not None:
            if championship.lower() == "wrc":
                championship = "World Rally Championship"
            elif championship.lower() == "erc":
                championship = "European Rally Championship"
            seasons_df = seasons_df[seasons_df["name"] == championship]

        if year is not None:
            seasons_df = seasons_df[seasons_df["year"] == year]
        return seasons_df

    def _getSeasons(self, championship=None, year=None, updateDB=False):
        """ The seasons feed is regularly updated throughout the season."""
        stub = f"seasons.json"
        json_data = self._WRC_RedBull_json(stub)
        seasons_df = DataFrame(json_data)
        if updateDB:
            self.dbfy(seasons_df, "seasons", pk="seasonId")

        seasons_df = self._getSeasonsSubQuery(seasons_df, championship, year)
        return seasons_df

    def getSeasons(self, championship=None, year=None, updateDB=False):
        if updateDB:
            self._getSeasons(championship, year, updateDB)

        q = "SELECT * FROM seasons;"
        seasons_df = read_sql(q, self.conn)

        return self._getSeasonsSubQuery(seasons_df, championship, year)

    # This datafeed is partial at the start of the season
    # and needs to be regularly updated
    def _getSeasonDetail(self, updateDB=False):
        stub = f"season-detail.json?seasonId={self.seasonId}"
        json_data = self._WRC_RedBull_json(stub)
        championships_df = DataFrame(json_data["championships"])
        seasonRounds_df = DataFrame(json_data["seasonRounds"])
        _seasonRounds_df = json_normalize(seasonRounds_df["event"])
        _cols = [
            "eventId",
            "countryId",
            "name",
            "slug",
            "location",
            "startDate",
            "finishDate",
            "timeZoneId",
            "timeZoneName",
            "timeZoneOffset",
            "surfaces",
            "organiserUrl",
            "categories",
            "mode",
            "trackingEventId",
            "clerkOfTheCourse",
            "stewards",
            "templateFilename",
            "country.countryId",
            "country.name",
            "country.iso2",
            "country.iso3",
            "seasonId",
            "order",
        ]
        seasonRounds_df = merge(
            _seasonRounds_df, seasonRounds_df[["eventId", "seasonId", "order"]]
        )[_cols]
        # TO DO - improve cleaning
        seasonRounds_df["name"] = seasonRounds_df["name"].str.strip()

        eligibilities_df = json_data["eligibilities"]

        if updateDB:
            self.dbfy(championships_df, "championship_lookup", pk="championshipId")
            self.dbfy(seasonRounds_df, "season_rounds", pk="eventId")

        return championships_df, seasonRounds_df, eligibilities_df

    def getSeasonRounds(self, updateDB=False):
        if updateDB:
            self._getSeasonDetail(updateDB)

        q = "SELECT * FROM season_rounds;"
        seasonRounds_df = read_sql(q, self.conn)

        return seasonRounds_df

    def getChampionships(self, updateDB=False):
        if updateDB:
            self._getSeasonDetail(updateDB)

        q = "SELECT * FROM championship_lookup;"
        championships_df = read_sql(q, self.conn)

        return championships_df

    def _getEventShakeDownTimes(self, updateDB=False):
        if not self.eventId:
            return
        stub = f"events/{self.eventId}/shakedowntimes.json?shakedownNumber=1"
        json_data = self._WRC_RedBull_json(stub)
        shakedownTimes_df = DataFrame(json_data)

        if updateDB:
            self.dbfy(shakedownTimes_df, "shakedown_times", pk="shakedownTimeId")

        return shakedownTimes_df

    def _getStages(self, updateDB=False):
        stub = f"events/{self.eventId}/stages.json"
        json_data = self._WRC_RedBull_json(stub)
        stages_df = DataFrame(json_data)
        stage_split_points_df = (
            stages_df[["splitPoints"]].explode("splitPoints").reset_index(drop=True)
        )
        stage_controls_df = json_normalize(
            stages_df[["controls"]].explode("controls").reset_index(drop=True)["controls"]
        )
        stage_split_points_df = json_normalize(
            stages_df[["splitPoints"]]
            .explode("splitPoints")
            .reset_index(drop=True)["splitPoints"]
        )
        stages_df.drop(columns=["splitPoints", "controls"], inplace=True)

        if updateDB:
            self.dbfy(stages_df, "stage_info", pk="stageId")
            self.dbfy(stage_split_points_df, "split_points", pk="splitPointId")
            self.dbfy(stage_controls_df, "stage_controls", pk="controlId")

        return stages_df, stage_split_points_df, stage_controls_df

    # TO DO  - set db
    def _getEvent(self, updateDB=False):
        """This also sets the self.rallyId"""
        stub = f"events/{self.eventId}.json"
        json_data = self._WRC_RedBull_json(stub)
        eventRallies_df = DataFrame(json_data["rallies"])
        eventClasses_df = DataFrame(json_data["eventClasses"])
        _data = {k: json_data[k] for k in json_data if k not in ["rallies", "eventClasses"]}
        eventData_df = DataFrame(_data)
        _event_df = eventRallies_df[eventRallies_df["isMain"] == True].iloc[0]

        self.rallyId = int(_event_df["rallyId"])
        self.itineraryId = int(_event_df["itineraryId"])

        return eventData_df, eventRallies_df, eventClasses_df

    def setEventByName(self, name=None, updateDB=False):
        if not name:
            return
        # Do this as a fuzzy search?
        q =f'SELECT eventId, name FROM season_rounds WHERE name="{name}";'
        r = read_sql(q, self.conn)
        # HACK TO DO this is a fudge
        if not r.empty:
            self.eventId = int(r.iloc[0]["eventId"])
            # Get the event info
            self._getEvent(updateDB=updateDB)

    def setStageByCode(self, stageCode=None, updateDB=False):
        if not stageCode:
            return
        # Do this as a fuzzy search?
        q =f'SELECT eventId, name FROM season_rounds WHERE name="{name}";'
        r = read_sql(q, self.conn)
        # HACK TO DO this is a fudge
        if not r.empty:
            self.eventId = int(r.iloc[0]["eventId"])
            # Get the event info
            self._getEvent(updateDB=updateDB)

    # TO DO  - set db
    # TO DO a way of setting self.controlId; also need strategies for invalidating Ids
    def _getControlTimes(self, updateDB=False):
        stub = f"events/{self.eventId}/controls/{self.controlId}/controlTimes.json"
        json_data = self._WRC_RedBull_json(stub)
        controlTimes_df = DataFrame(json_data)

        if updateDB:
            self.dbfy(controlTimes_df, "controltimes", pk="controlTimeId")

        return controlTimes_df

    #  TO DO set stageId somehow
    def _getSplitTimes(self, updateDB=False):
        stub = f"events/{self.eventId}/stages/{self.stageId}/splittimes.json?rallyId={self.rallyId}"
        json_data = self._WRC_RedBull_json(stub)
        splitTimes_df = DataFrame(json_data)
        splitTimes_df["eventId"] = self.eventId
        splitTimes_df["stageId"] = self.stageId

        if updateDB:
            self.dbfy(splitTimes_df, "split_times", pk="splitPointTimeId")

        return splitTimes_df

    # TO DO set stageId somehow
    def _getStageResults(self, updateDB=False):
        """ This is the overall result at the end of the stage. TO DO CHECK """
        stub = f"events/{self.eventId}/stages/{self.stageId}/results.json?rallyId={self.rallyId}"
        json_data = self._WRC_RedBull_json(stub)
        stageResults_df = DataFrame(json_data)
        stageResults_df["stageId"] = self.stageId
        stageResults_df["eventId"] = self.eventId

        if updateDB:
            self.dbfy(stageResults_df, "stage_overall", pk=("stageId", "entryId"))

        return stageResults_df

    def _getStageWinners(self, updateDB=False):
        stub = f"events/{self.eventId}/rallies/{self.rallyId}/stagewinners.json"
        json_data = self._WRC_RedBull_json(stub)
        stagewinners_df = DataFrame(json_data)
        stagewinners_df["eventId"] = self.eventId
        stagewinners_df["rallyId"] = self.rallyId
        if updateDB:
            self.dbfy(stagewinners_df, "stagewinners", pk="stageId")

        return stagewinners_df

    def _getRetirements(self, updateDB=False):
        if not self.eventId:
            return
        stub = f"events/{self.eventId}/retirements.json"
        json_data = self._WRC_RedBull_json(stub)
        retirements_df = DataFrame(json_data)
        retirements_df["eventId"] = self.eventId
        if updateDB:
            self.dbfy(retirements_df, "retirements", pk="retirementId")
        return retirements_df

    def _getPenalties(self, updateDB=False):
        if not self.eventId:
            return
        stub = f"events/{self.eventId}/penalties.json"
        json_data = self._WRC_RedBull_json(stub)
        penalties_df = DataFrame(json_data)
        penalties_df["eventId"] = self.eventId
        if updateDB:
            self.dbfy(penalties_df, "penalties", pk="penaltyId")
        return penalties_df

    def getStageWinners(self):
        sql = f"""SELECT * FROM stagewinners WHERE eventId={self.eventId} AND rallyId={self.rallyId};"""
        r = read_sql(sql, self.conn)
        # Hack to poll API if empty
        if r.empty:
            self._getStageWinners(updateDB=True)
            r = read_sql(sql, self.conn)
        return r

    # TO DO - offer more search limits
    def getRetirements(self, updateDB=False):
        if not self.eventId:
            return
        sql = f"""SELECT * FROM retirements WHERE eventId={self.eventId};"""
        r = read_sql(sql, self.conn)
        # Hack to poll API if empty
        if r.empty:
            self._getRetirements(updateDB=True)
            r = read_sql(sql, self.conn)
        return r

    # TO DO - offer more search limits
    def getPenalties(self):
        if not self.eventId:
            return
        sql = f"""SELECT * FROM penalties WHERE eventId={self.eventId};"""
        r = read_sql(sql, self.conn)
        # Hack to poll API if empty
        if r.empty:
            self._getPenalties(updateDB=True)
            r = read_sql(sql, self.conn)
        return r

    def query(self, sql):
        r = read_sql(sql, self.conn)
        return r

In [6]:
wrc = WRCLiveTimingAPIClientV2(newDB=True)

INFO:__main__:Initialising the database...
INFO:__main__:Creating new db tables...


https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/seasons.json
https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/seasons.json
https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/season-detail.json?seasonId=34


INFO:__main__:Updating championship_lookup...
INFO:__main__:Updating season_rounds...


In [7]:
wrc.setEventByName("Rally Sweden")
wrc.eventId

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535.json


535

In [8]:
wrc.getPenalties()

INFO:__main__:Updating penalties...


https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/penalties.json


,controlId,entryId,penaltyDuration,penaltyDurationMs,penaltyId,reason,eventId
0,35212,54834,PT10S,10000,7209,1 MIN LATE,535
1,35255,54834,PT1M,60000,7211,1 MIN EARLY,535
2,35213,54807,PT4M30S,270000,7212,27 MINS LATE,535
3,35213,54835,PT2M20S,140000,7213,14 MINS LATE,535
4,35268,54792,PT10S,10000,7216,FALSE START,535
5,35216,54837,PT10S,10000,7217,1 MIN LATE,535
6,35222,54786,PT2M,120000,7218,12 MINS LATE,535
7,35224,54826,PT10S,10000,7219,1 MIN LATE,535


In [9]:
wrc.getStageWinners()

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/rallies/583/stagewinners.json


INFO:__main__:Updating stagewinners...


,elapsedDuration,elapsedDurationMs,entryId,stageId,stageName,eventId,rallyId
0,00:03:21.6000000,201600,54778,8330,Umeå Sprint 1,535,583
1,00:13:58.8000000,838800,54778,8331,Bygdsiljum 1,535,583
2,00:10:43.8000000,643800,54780,8332,Andersvattnet 1,535,583
3,00:05:54.6000000,354600,54780,8333,Bäck 1,535,583
4,00:13:50.1000000,830100,54781,8334,Bygdsiljum 2,535,583
5,00:10:49.2000000,649200,54779,8335,Andersvattnet 2,535,583
6,00:06:02.5000000,362500,54777,8336,Bäck 2,535,583
7,00:03:29.9000000,209900,54778,8337,Umeå Sprint 2,535,583
8,00:09:00.5000000,540500,54782,8338,Vännäs 1,535,583
9,00:06:37.5000000,397500,54778,8339,Sarsjöliden 1,535,583


In [10]:
stages_df, stage_split_points_df, stage_controls_df = wrc._getStages()
stages_df, stage_split_points_df, stage_controls_df
stage_controls_df

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages.json


,controlId,eventId,stageId,type,code,location,timingPrecision,distance,targetDuration,targetDurationMs,firstCarDueDateTime,firstCarDueDateTimeLocal,status,controlPenalties,roundingPolicy,locked,bogey,bogeyMs
0,35228,535,8330,TimeControl,TC1,Umeå Sprint,Minute,4.97,00:20:00,1200000.0,2025-02-13T18:00:00,None,Completed,All,NoRounding,True,None,None
1,35229,535,8330,StageStart,SS1,Umeå Sprint 1,Minute,5.16,00:05:00,300000.0,2025-02-13T18:05:00,None,Completed,None,RoundToClosestMinute,True,None,None
2,35230,535,8330,FlyingFinish,SF1,Umeå Sprint 1,Tenth,NaN,None,NaN,None,None,Completed,None,NoRounding,True,None,None
3,35231,535,8331,TimeControl,TC2,Bygdsiljum,Minute,61.13,01:10:00,4200000.0,2025-02-14T08:15:00,None,Completed,All,NoRounding,True,None,None
4,35232,535,8331,StageStart,SS2,Bygdsiljum 1,Minute,28.27,00:03:00,180000.0,2025-02-14T08:18:00,None,Completed,None,RoundToClosestMinute,True,None,None
5,35233,535,8331,FlyingFinish,SF2,Bygdsiljum 1,Tenth,NaN,None,NaN,None,None,Completed,None,NoRounding,True,None,None
6,35234,535,8332,TimeControl,TC3,Andersvattnet,Minute,22.00,00:58:00,3480000.0,2025-02-14T09:16:00,None,Completed,All,NoRounding,True,None,None
7,35235,535,8332,StageStart,SS3,Andersvattnet 1,Minute,20.51,00:03:00,180000.0,2025-02-14T09:19:00,None,Completed,None,RoundToClosestMinute,True,None,None
8,35236,535,8332,FlyingFinish,SF3,Andersvattnet 1,Tenth,NaN,None,NaN,None,None,Completed,None,NoRounding,True,None,None
9,35237,535,8333,TimeControl,TC4,Bäck,Minute,41.01,01:05:00,3900000.0,2025-02-14T10:24:00,None,Completed,All,NoRounding,True,None,None


In [10]:
#
q = "SELECT name from season_rounds"
q = f'SELECT eventId, name FROM season_rounds WHERE name="Rally Sweden";'
wrc.query(q).iloc[0]["eventId"]

np.int64(535)

In [11]:
wrc.getSeasons()

,seasonId,name,year


In [12]:
wrc.getChampionships()

,championshipId,name,seasonId,type,fieldOneDescription,fieldTwoDescription,fieldThreeDescription,fieldFiveDescription,fieldFourDescription
0,287,FIA World Rally Championship for Drivers,34,Drivers,FirstName,LastName,CountryISO3,TyreManufacturer,Manufacturer
1,288,FIA World Rally Championship for Co-Drivers,34,Codrivers,FirstName,LastName,CountryISO3,TyreManufacturer,Manufacturer
2,289,FIA World Rally Championship for Manufacturers,34,Manufacturer,Name,Manufacturer,LogoFileName,,
3,290,FIA World Rally Championship for Teams,34,Manufacturer,Name,Manufacturer,LogoFileName,,
4,291,FIA WRC2 Championship for Drivers,34,DriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer
5,292,FIA WRC2 Championship for Co-Drivers,34,CodriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer
6,293,FIA WRC2 Challenger Championship for Drivers,34,DriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer
7,294,FIA WRC2 Challenger Championship for Co-Drivers,34,CodriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer
8,295,FIA WRC2 Championship for Teams,34,Manufacturer,Name,Manufacturer,LogoFileName,,
9,296,FIA WRC Masters Cup for Drivers,34,DriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer


In [13]:
# From the seasons we can search on year and name
# Names: "World Rally Championship", "European Rally Championship"
# Return: seasonId (e.g World Rally Championship 2025: seasonId=34)
#
wrc.getSeasonRounds()

,categories,clerkOfTheCourse,country.countryId,country.iso2,country.iso3,country.name,countryId,eventId,finishDate,location,...,startDate,stewards,surfaces,templateFilename,timeZoneId,timeZoneName,timeZoneOffset,trackingEventId,order,seasonId
0,None,Romain PUGLIESE,147,MC,MCO,Monaco,147,534,2025-01-26,"Gap, France",...,2025-01-23,Tanja GEILHAUSEN Edoardo DELLEANI ...,Mixed,results-report-templates/e352213e-0a5d-443f-b9...,Europe/Monaco,(UTC+01:00) Central European Time (Monaco),60,4208,1,34
1,None,Stig Rune Kjernsli,215,SE,SWE,Sweden,215,535,2025-02-16,"Umeå, Västerbotten County",...,2025-02-13,Mazen Al-Hilli Mathieu Remmerie ...,Snow,results-report-templates/99d0742c-4b8a-4bc9-ab...,Europe/Stockholm,(UTC+01:00) Central European Time (Stockholm),60,4209,2,34
2,None,George Mwangi,116,KE,KEN,Kenya,116,536,2025-03-23,Nairobi,...,2025-03-20,Tanja Geilhausen Alice Scutellà ...,Gravel,results-report-templates/fde0030f-74a1-48a6-ba...,Africa/Nairobi,(UTC+03:00) East Africa Time (Nairobi),180,4210,3,34
3,None,None,209,ES,ESP,Spain,209,538,2025-04-27,"Las Palmas, Canarias",...,2025-04-24,None,Tarmac,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4211,4,34
4,None,None,178,PT,PRT,Portugal,178,540,2025-05-18,"Matosinhos, Porto",...,2025-05-15,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4212,5,34
5,None,None,110,IT,ITA,Italy,110,542,2025-06-08,"Alghero, Sardinia",...,2025-06-05,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4213,6,34
6,None,None,86,GR,GRC,Greece,86,544,2025-06-29,"Lamia, Central Greece",...,2025-06-26,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4214,7,34
7,None,None,70,EE,EST,Estonia,70,546,2025-07-20,Tartu,...,2025-07-17,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4215,8,34
8,None,None,75,FI,FIN,Finland,75,547,2025-08-03,Jyväskylä,...,2025-07-31,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4216,9,34
9,None,None,173,PY,PRY,Paraguay,173,549,2025-08-31,Itapúa,...,2025-08-28,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4217,10,34


In [ ]:
# TO DO SEE ALSO eg https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/536/championships.json
#

def _getChampionshipDetail(championshipId=None, seasonId=None, championship="WRC", category="Drivers"):
    # If championshipId is None, try to find a championship Id
    if not championshipId:
        # Category: Drivers, Co-Drivers, Manufacturers
        # TO DO - we can make this more robust, eg on case, hyphenation
        categories = ["Drivers", "Co-Drivers"]
        if championship.lower() in ["wrc", "wrc2"]:
            categories.append("Teams")
        if championship.lower() in ["wrc"]:
            categories.append("Manufacturers")

        category = category if category in categories else "Drivers"
        # Use WRC drivers das the default
        _championship = "FIA World Rally Championship for Drivers"
        if championship.lower()=="wrc":
            _championship = f"FIA World Rally Championship for {category}"
        elif championship.lower() == "wrc2":
            _championship = f"FIA WRC2 Championship for {category}"
        elif championship.lower() == "wrc3":
            _championship = f"FIA WRC3 Championship for {category}"
        elif championship.lower() == "jwrc":
            _championship = f"FIA Junior WRC Championship for {category}"
        elif championship.lower() == "challenger":
            _championship = f"FIA WRC2 Challenger Championship for {category}"
        elif championship.lower() == "masters":
            _championship = f"FIA WRC Masters Cup for {category}"
        seasonId, championships_df, _, _ = getSeasonDetail(seasonId=seasonId)
        championshipId = championships_df[
            championships_df["name"] == _championship
        ].iloc[0]["championshipId"]

    if seasonId is None:
        seasonId = _getSeasons("WRC", 2025).iloc[0]["seasonId"]

    stub = f"championship-detail.json?championshipId={championshipId}&seasonId={seasonId}"
    json_data = wrc._WRC_RedBull_json(stub)
    # championships_df = DataFrame(json_data["championships"])
    # seasonRounds_df = DataFrame(json_data["seasonRounds"])
    # eligibilities_df = json_data["eligibilities"]
    # return championships_df, seasonRounds_df, eligibilities_df
    rounds = [r["event"] for r in json_data["championshipRounds"]]
    championshipRounds_df = DataFrame(rounds)
    championshipCountries_df = json_normalize(championshipRounds_df["country"]).drop_duplicates()
    championshipRounds_df.drop(columns=["country"])

    renamers = {
        k.replace("Description", ""): json_data[k]
        for k in json_data.keys()
        if k.startswith("field") and k.endswith("Description")
    }

    _e = json_data["championshipEntries"]
    championshipEntries_df =  DataFrame(_e)
    championshipEntries_df.rename(columns=renamers, inplace=True)

    return championshipRounds_df, championshipEntries_df, championshipCountries_df


d1, d2, d3= _getChampionshipDetail()
d1.head(3), d2.head(), d3.head()

NameError: name 'getSeasonDetail' is not defined

In [ ]:
def _getChampionshipOverallResults(championshipId=287, seasonId=34):
    stub = f"championship-overall-results.json?championshipId={championshipId}&seasonId={seasonId}"
    json_data = wrc._WRC_RedBull_json(stub)

    _entryResultsOverall = []
    _entryResultsByRound = []

    for entry in json_data["entryResults"]:
        _entry={}
        for k in ["championshipEntryId", "overallPosition", "overallPoints"]:
            _entry[k] = entry[k]
        _entryResultsOverall.append(_entry)
        _entryResultsByRound.extend(entry["roundResults"])

    entryResultsOverall_df = DataFrame(_entryResultsOverall)
    entryResultsByRound_df = DataFrame(_entryResultsByRound)

    return entryResultsOverall_df, entryResultsByRound_df, json_data

a, b, c = _getChampionshipOverallResults()
a,b

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/championship-overall-results.json?championshipId=287&seasonId=34


(    championshipEntryId  overallPosition  overallPoints
 0                  6700                1             61
 1                  6699                2             33
 2                  6698                3             31
 3                  6702                4             29
 4                  6697                5             26
 5                  6784                6             25
 6                  6701                7             21
 7                  6836                8              8
 8                  6782                9              6
 9                  6703               10              6
 10                 6704               11              4
 11                 6783               12              4
 12                 6837               13              2
 13                 6705               14              2
 14                 6706               15              1
 15                 6838               16              1
 16                 6839       

In [28]:
eventId = 535 # safari 536, sweden 535

In [29]:
def getEventGroups(eventId):
    stub = f"events/{eventId}/groups.json"
    json_data = wrc._WRC_RedBull_json(stub)
    eventGroups_df = DataFrame(json_data)
    return eventGroups_df

getEventGroups(eventId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/groups.json


,groupId,name
0,152,Rally1
1,98,Rally2
2,121,Rally3
3,100,Rally5


In [ ]:

getEventShakeDownTimes(eventId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/shakedowntimes.json?shakedownNumber=1


,shakedownTimeId,eventId,entryId,runNumber,shakedownNumber,runDuration,runDurationMs
0,18735,535,54777,1,1,PT2M10.805S,130805
1,18754,535,54777,2,1,PT2M11.12S,131120
2,18791,535,54777,3,1,PT2M9.62S,129620
3,18824,535,54777,4,1,PT2M6.527S,126527
4,18731,535,54778,1,1,PT2M8.792S,128792
...,...,...,...,...,...,...,...
148,18878,535,54836,1,1,PT2M47.694S,167694
149,18863,535,54837,1,1,PT2M52.537S,172537
150,18855,535,54838,1,1,PT3M15.317S,195317
151,18867,535,54838,2,1,PT3M7.743S,187743


In [ ]:


rallyId, itineraryId, eventData_df, eventRallies_df, eventClasses_df = getEvent(eventId)
# rallyId = 584  # Safari WRC on eventId = 536 itineraryId = 1155
rallyId, itineraryId,  eventData_df, eventRallies_df, eventClasses_df

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535.json


(583,
 1137,
            eventId  countryId  eventProfileId country          name  \
 countryId      535        215               2     215  Rally Sweden   
 name           535        215               2  Sweden  Rally Sweden   
 iso2           535        215               2      SE  Rally Sweden   
 iso3           535        215               2     SWE  Rally Sweden   
 
                       slug                   location   startDate  finishDate  \
 countryId  2025-wrc-sweden  Umeå, Västerbotten County  2025-02-13  2025-02-16   
 name       2025-wrc-sweden  Umeå, Västerbotten County  2025-02-13  2025-02-16   
 iso2       2025-wrc-sweden  Umeå, Västerbotten County  2025-02-13  2025-02-16   
 iso3       2025-wrc-sweden  Umeå, Västerbotten County  2025-02-13  2025-02-16   
 
                  timeZoneId  ...    clerkOfTheCourse  \
 countryId  Europe/Stockholm  ...  Stig Rune Kjernsli   
 name       Europe/Stockholm  ...  Stig Rune Kjernsli   
 iso2       Europe/Stockholm  ...  Stig Ru

In [ ]:


def getEntries(eventId, rallyId):
    stub = f"events/{eventId}/rallies/{rallyId}/entries.json"
    json_data = wrc._WRC_RedBull_json(stub)
    entries_df = DataFrame(json_data)
    drivers_df = json_normalize(entries_df["driver"])
    codrivers_df = json_normalize(entries_df["codriver"])
    manufacturers_df = json_normalize(entries_df["manufacturer"]).drop_duplicates()
    entrants_df = json_normalize(entries_df["entrant"]).drop_duplicates()
    entryGroups_df = json_normalize(entries_df["group"]).drop_duplicates()
    eventClasses_df = json_normalize(
        entries_df.explode("eventClasses")["eventClasses"]
    ).drop_duplicates()

    entries_df.drop(columns=["driver", "codriver", "manufacturer", "entrant", "group", "eventClasses"], inplace=True)
    return (
        entries_df,
        drivers_df,
        codrivers_df,
        manufacturers_df,
        entrants_df,
        entryGroups_df,
        eventClasses_df,
    )

getEntries(eventId, rallyId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/rallies/583/entries.json


(   tags  entryId  eventId  driverId  codriverId  manufacturerId  entrantId  \
 0    []    54777      535       762         851              33        166   
 1    []    54778      535       534         553              84         91   
 2    []    54779      535       524         525              33        166   
 3    []    54780      535      4676        4945              33        166   
 4    []    54781      535       785        5507              84         91   
 ..  ...      ...      ...       ...         ...             ...        ...   
 57   []    54834      535     14234       17360              33       1148   
 58   []    54835      535     17361       17362              77       3801   
 59   []    54836      535      5525       17363              33       1380   
 60   []    54837      535       610         611              77       1379   
 61   []    54838      535       626        5545              69        572   
 
     groupId identifier     vehicleModel  entryLis

In [33]:
from pandas import concat

def getEventItineraries(eventId, itineraryId):
    stub = f"events/{eventId}/itineraries/{itineraryId}.json"
    json_data = wrc._WRC_RedBull_json(stub)
    itineraryLegs_df = DataFrame(json_data["itineraryLegs"])
    itinerarySections_df = itineraryLegs_df.explode("itinerarySections")

    itinerarySections2_df = json_normalize(itinerarySections_df["itinerarySections"])
    itineraryControls_df = itinerarySections2_df.explode("controls").reset_index(drop=True)
    _itineraryControls_df = json_normalize(itineraryControls_df['controls'])
    itineraryControls_df = concat(
        [itineraryControls_df.drop("controls", axis=1), _itineraryControls_df], axis=1
    )
    itineraryStages_df = itinerarySections2_df.explode("stages").reset_index(drop=True)
    _itineraryStages_df = json_normalize(itineraryStages_df["stages"])
    itineraryStages_df = concat(
        [itineraryStages_df.drop("stages", axis=1), _itineraryStages_df], axis=1
    )
    itineraryLegs_df.drop(columns=["itinerarySections"], inplace=True)
    itinerarySections_df.drop(columns=["itinerarySections"], inplace=True)
    itineraryControls_df.drop(columns=["stages"], inplace=True)
    itineraryStages_df.drop(columns=["controls"], inplace=True)
    return (
        itineraryLegs_df,
        itinerarySections_df,
        itineraryControls_df,
        itineraryStages_df,
    )

itineraryLegs_df,itinerarySections_df,itineraryControls_df,itineraryStages_df = getEventItineraries(
    eventId, itineraryId
)
itineraryControls_df, itineraryStages_df

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/itineraries/1137.json


(    itinerarySectionId  itineraryLegId  order        name  controlId  eventId  \
 0                 3635            1506      1   Section 1      35201      535   
 1                 3635            1506      1   Section 1      35228      535   
 2                 3635            1506      1   Section 1      35229      535   
 3                 3635            1506      1   Section 1      35230      535   
 4                 3635            1506      1   Section 1      35202      535   
 ..                 ...             ...    ...         ...        ...      ...   
 76                3644            1509     10  Section 10      35226      535   
 77                3644            1509     10  Section 10      35279      535   
 78                3644            1509     10  Section 10      35280      535   
 79                3644            1509     10  Section 10      35281      535   
 80                3644            1509     10  Section 10      35227      535   
 
     stageId  

In [ ]:

getStages(eventId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages.json


(    stageId  eventId  number                  name  distance     status  \
 0      8330      535       1         Umeå Sprint 1      5.16  Completed   
 1      8331      535       2          Bygdsiljum 1     28.27  Completed   
 2      8332      535       3       Andersvattnet 1     20.51  Completed   
 3      8333      535       4                Bäck 1     10.80  Completed   
 4      8334      535       5          Bygdsiljum 2     28.27  Completed   
 5      8335      535       6       Andersvattnet 2     20.51  Completed   
 6      8336      535       7                Bäck 2     10.80  Completed   
 7      8337      535       8         Umeå Sprint 2      5.16  Completed   
 8      8338      535       9              Vännäs 1     15.65  Completed   
 9      8339      535      10         Sarsjöliden 1     14.23  Completed   
 10     8340      535      11            Kolksele 1     16.06  Completed   
 11     8341      535      12              Vännäs 2     15.65  Completed   
 12     8342

In [ ]:


getStageWinners(eventId, rallyId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/rallies/583/stagewinners.json


,stageId,entryId,stageName,elapsedDurationMs,elapsedDuration
0,8330,54778,Umeå Sprint 1,201600,00:03:21.6000000
1,8331,54778,Bygdsiljum 1,838800,00:13:58.8000000
2,8332,54780,Andersvattnet 1,643800,00:10:43.8000000
3,8333,54780,Bäck 1,354600,00:05:54.6000000
4,8334,54781,Bygdsiljum 2,830100,00:13:50.1000000
5,8335,54779,Andersvattnet 2,649200,00:10:49.2000000
6,8336,54777,Bäck 2,362500,00:06:02.5000000
7,8337,54778,Umeå Sprint 2,209900,00:03:29.9000000
8,8338,54782,Vännäs 1,540500,00:09:00.5000000
9,8339,54778,Sarsjöliden 1,397500,00:06:37.5000000


In [ ]:


getPenalties(eventId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/penalties.json


,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,7209,35212,54834,10000,PT10S,1 MIN LATE
1,7211,35255,54834,60000,PT1M,1 MIN EARLY
2,7212,35213,54807,270000,PT4M30S,27 MINS LATE
3,7213,35213,54835,140000,PT2M20S,14 MINS LATE
4,7216,35268,54792,10000,PT10S,FALSE START
5,7217,35216,54837,10000,PT10S,1 MIN LATE
6,7218,35222,54786,120000,PT2M,12 MINS LATE
7,7219,35224,54826,10000,PT10S,1 MIN LATE


In [ ]:


_getRetirements(eventId)

NameError: name 'eventId' is not defined

In [ ]:


_getStageResults(535, 583, 8332)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages/8332/results.json?rallyId=583


,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev
0,54778,1686000,PT28M6S,0,PT0S,1686000,PT28M6S,1,0,PT0S,0,PT0S
1,54779,1689900,PT28M9.9S,0,PT0S,1689900,PT28M9.9S,2,3900,PT3.9S,3900,PT3.9S
2,54780,1691800,PT28M11.8S,0,PT0S,1691800,PT28M11.8S,3,5800,PT5.8S,1900,PT1.9S
3,54781,1693700,PT28M13.7S,0,PT0S,1693700,PT28M13.7S,4,7700,PT7.7S,1900,PT1.9S
4,54777,1702800,PT28M22.8S,0,PT0S,1702800,PT28M22.8S,5,16800,PT16.8S,9100,PT9.1S
5,54782,1702900,PT28M22.9S,0,PT0S,1702900,PT28M22.9S,6,16900,PT16.9S,100,PT0.1S
6,54786,1718900,PT28M38.9S,0,PT0S,1718900,PT28M38.9S,7,32900,PT32.9S,16000,PT16S
7,54785,1719800,PT28M39.8S,0,PT0S,1719800,PT28M39.8S,8,33800,PT33.8S,900,PT0.9S
8,54784,1741700,PT29M1.7S,0,PT0S,1741700,PT29M1.7S,9,55700,PT55.7S,21900,PT21.9S
9,54783,1744300,PT29M4.3S,0,PT0S,1744300,PT29M4.3S,10,58300,PT58.3S,2600,PT2.6S


In [ ]:


getControlTimes(534, 34754)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/534/controls/34754/controlTimes.json


,controlTimeId,controlId,entryId,dueDateTime,dueDateTimeLocal,actualDateTime,actualDateTimeLocal,absoluteActualDateTime,absoluteActualDateTimeLocal,source,status
0,1407418,34754,54668,2025-01-24T08:28:00,2025-01-24T09:28:00+01:00,2025-01-24T08:28:00,2025-01-24T09:28:00+01:00,2025-01-24T08:28:00,2025-01-24T09:28:00+01:00,Default,Completed
1,1407444,34754,54669,2025-01-24T08:31:00,2025-01-24T09:31:00+01:00,2025-01-24T08:31:00,2025-01-24T09:31:00+01:00,2025-01-24T08:31:00,2025-01-24T09:31:00+01:00,Default,Completed
2,1407470,34754,54670,2025-01-24T08:34:00,2025-01-24T09:34:00+01:00,2025-01-24T08:34:00,2025-01-24T09:34:00+01:00,2025-01-24T08:34:00,2025-01-24T09:34:00+01:00,Default,Completed
3,1407496,34754,54671,2025-01-24T08:37:00,2025-01-24T09:37:00+01:00,2025-01-24T08:37:00,2025-01-24T09:37:00+01:00,2025-01-24T08:37:00,2025-01-24T09:37:00+01:00,Default,Completed
4,1407522,34754,54672,2025-01-24T08:40:00,2025-01-24T09:40:00+01:00,2025-01-24T08:40:00,2025-01-24T09:40:00+01:00,2025-01-24T08:40:00,2025-01-24T09:40:00+01:00,Default,Completed
...,...,...,...,...,...,...,...,...,...,...,...
63,1408640,34754,54733,2025-01-24T10:04:00,2025-01-24T11:04:00+01:00,2025-01-24T10:04:00,2025-01-24T11:04:00+01:00,2025-01-24T10:04:00,2025-01-24T11:04:00+01:00,Default,Completed
64,1409030,34754,54734,2025-01-24T10:19:00,2025-01-24T11:19:00+01:00,2025-01-24T10:19:00,2025-01-24T11:19:00+01:00,2025-01-24T10:19:00,2025-01-24T11:19:00+01:00,Default,Completed
65,1409082,34754,54735,2025-01-24T10:21:00,2025-01-24T11:21:00+01:00,2025-01-24T10:21:00,2025-01-24T11:21:00+01:00,2025-01-24T10:21:00,2025-01-24T11:21:00+01:00,Default,Completed
66,1408900,34754,54736,2025-01-24T10:14:00,2025-01-24T11:14:00+01:00,2025-01-24T10:14:00,2025-01-24T11:14:00+01:00,2025-01-24T10:14:00,2025-01-24T11:14:00+01:00,Default,Completed


In [ ]:


getSplitTimes(534, 582, 8213)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/534/stages/8213/splittimes.json?rallyId=582


,startDateTime,startDateTimeLocal,stageTimeDurationMs,stageTimeDuration,splitPointTimeId,splitPointId,entryId,elapsedDurationMs,elapsedDuration,splitDateTime,splitDateTimeLocal
0,2025-01-24T08:31:00,2025-01-24T09:31:00+01:00,720600.0,00:12:00.6000000,467539,13346,54668,193500,PT3M13.5S,2025-01-24T08:34:13.5,2025-01-24T09:34:13.5+01:00
1,2025-01-24T08:34:00,2025-01-24T09:34:00+01:00,717800.0,00:11:57.8000000,467542,13346,54669,194700,PT3M14.7S,2025-01-24T08:37:14.7,2025-01-24T09:37:14.7+01:00
2,2025-01-24T08:37:00,2025-01-24T09:37:00+01:00,705000.0,00:11:45,467546,13346,54670,189800,PT3M9.8S,2025-01-24T08:40:09.8,2025-01-24T09:40:09.8+01:00
3,2025-01-24T08:40:00,2025-01-24T09:40:00+01:00,711000.0,00:11:51,467554,13346,54671,190700,PT3M10.7S,2025-01-24T08:43:10.7,2025-01-24T09:43:10.7+01:00
4,2025-01-24T08:43:00,2025-01-24T09:43:00+01:00,706400.0,00:11:46.4000000,467563,13346,54672,189200,PT3M9.2S,2025-01-24T08:46:09.2,2025-01-24T09:46:09.2+01:00
...,...,...,...,...,...,...,...,...,...,...,...
457,2025-01-24T10:22:00,2025-01-24T11:22:00+01:00,894000.0,00:14:54,467982,13353,54734,641300,PT10M41.3S,2025-01-24T10:32:41.3,2025-01-24T11:32:41.3+01:00
458,2025-01-24T10:23:00,2025-01-24T11:23:00+01:00,933700.0,00:15:33.7000000,467986,13353,54710,680000,PT11M20S,2025-01-24T10:34:20,2025-01-24T11:34:20+01:00
459,2025-01-24T10:24:00,2025-01-24T11:24:00+01:00,898800.0,00:14:58.8000000,467988,13353,54735,651200,PT10M51.2S,2025-01-24T10:34:51.2,2025-01-24T11:34:51.2+01:00
460,2025-01-24T10:25:00,2025-01-24T11:25:00+01:00,961200.0,00:16:01.2000000,467993,13353,54730,701500,PT11M41.5S,2025-01-24T10:36:41.5,2025-01-24T11:36:41.5+01:00


## Comparative testing

Compare old and new dataframes.


In [24]:
from wrc_rallydj.livetiming_api import WRCLiveTimingAPIClient

wrc_old = WRCLiveTimingAPIClient(use_cache=True, backend="memory", expire_after=60000)
wrc_old.initialise(eventName="WRC Rally Sweden")

Trying       id         guid                           title                location  \
0   K1Lg  WRC_2025_01              Rallye Monte-Carlo                  Monaco   
1   skC3  WRC_2025_02                WRC Rally Sweden                  sweden   
2   G2NQ  WRC_2025_03          WRC Safari Rally Kenya         Naivasha, Kenya   
3   zhuM  WRC_2025_04        WRC Rally Islas Canarias   Islas Canarias, Spain   
4   edGQ  WRC_2025_05  WRC Vodafone Rally de Portugal       Matosinhos, Porto   
5   k5hf  WRC_2025_06       WRC Rally Italia Sardegna         Olbia, Sardinia   
6   eaYx  WRC_2025_07  WRC EKO Acropolis Rally Greece                   Lamia   
7   aIpJ  WRC_2025_08         WRC Delfi Rally Estonia                 Estonia   
8   CaGI  WRC_2025_09         WRC Secto Rally Finland               Jyväskylä   
9   nMcx  WRC_2025_10          WRC Rally del Paraguay                Paraguay   
10  XpJ0  WRC_2025_11         WRC Rally Chile Bio Bío              Concepcion   
11  nQmX  WRC_2025_12

In [26]:
xx= wrc_old.getPenalties()
xx.columns, xx.head()

(Index(['id', 'carNo', 'driverId', 'driverCountry', 'driverCountryImage',
        'driver', 'coDriverId', 'coDriverCountry', 'coDriverCountryImage',
        'coDriver', 'teamId', 'team/car', 'teamName', 'teamLogo', 'eligibility',
        'groupClass', 'rallyId', 'entryId', 'controlId', 'penaltyTime',
        'penaltyDuration', 'penaltyDurationMs', 'reason', 'control'],
       dtype='object'),
      id carNo                              driverId driverCountry  \
 0  7208   #66  917a7353-c28f-5fa4-b9f2-3afc0f6993e9       Belgium   
 1  7209   #70  d075fa9c-91dd-58a3-9a14-c24fe8fdb7f2          Peru   
 2  7210   #51  bb8301ac-ba00-57ff-ad20-0e03a535cb0c    Greece IOC   
 3  7211   #70  d075fa9c-91dd-58a3-9a14-c24fe8fdb7f2          Peru   
 4  7212   #41  5420dcab-71e0-5b63-a8f5-65ce359ed1e0        Turkey   
 
   driverCountryImage              driver  \
 0      Flags/BEL.png       Lyssia BAUDET   
 1      Flags/PER.png      Jorge MARTÍNEZ   
 2      Flags/GRE.png  Georgios VASILAKIS   
 3